In [18]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim
from gensim.test.utils import common_texts
import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
import torch

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} \
         if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) \
               if device != "cpu" else default_collate

with open('embedding_data.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)


n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import OneHotEncoder
label_encoder=OneHotEncoder(sparse_output=False)

# -


# X = embed_df['vectors'] # Word Embeddings

# +
test_size = 0.2
val_size = 0.2
random_state =699


# +
# with open('embedding_data.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)

data = U.load_file('data_vFF.pkl', 'pkl', config['DATADIR'])


In [19]:

embed_df = pd.DataFrame(data)

embed_df['join_text'] = embed_df['text'].apply(' '.join)



In [20]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
X = embed_df['passage_key']

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)




/Users/stephentoner/miniconda3/envs/si699proj/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [42]:
embed_df['label'] = y

train_set = set(X_train)
val_set = set(X_val)
test_set = set(X_test)

In [43]:
train = embed_df[embed_df['passage_key'].apply(lambda S: S in train_set)]
val = embed_df[embed_df['passage_key'].apply(lambda S: S in val_set)]
test = embed_df[embed_df['passage_key'].apply(lambda S: S in test_set)]


In [35]:
train

,author_id,author_name,book_id,gutenbergbookid,title,text,text_lines,authoryearofbirth,authoryearofdeath,downloads,subjects,topic,Sub_A,Sub_B,Sub_C,str_text_lines,passage_key,join_text,label


In [23]:
from datasets import Dataset

In [44]:
ds = {
      'train' :  Dataset.from_dict(train.to_dict()),
      'val' :  Dataset.from_dict(val.to_dict()),
      'test' :  Dataset.from_dict(test.to_dict()),
      }

In [26]:
embed_df.columns

Index(['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines', 'authoryearofbirth', 'authoryearofdeath',
       'downloads', 'subjects', 'topic', 'Sub_A', 'Sub_B', 'Sub_C',
       'str_text_lines', 'passage_key', 'join_text', 'label'],
      dtype='object')

In [47]:
type(embed_df.join_text.iloc[0])

str

In [48]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.integrations import TensorBoardCallback
from transformers.trainer_callback import EarlyStoppingCallback
from torch.utils.data import DataLoader

BASE_MODEL = "allenai/longformer-base-4096"
# BASE_MODEL = "lreN5bs16" # Learning Rate 2e-5, batch size 16
LEARNING_RATE = 2e-5
MAX_LENGTH = 400
BATCH_SIZE = 16
EPOCHS = 3

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, 
                                                           num_labels=n_classes,
                                                           ignore_mismatched_sizes=True)

def preprocess_function(examples, test = False):
    # if not test:
    label = examples["label"] 
    examples = tokenizer(examples["text"],
                        truncation=True, 
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors='pt')
    
    for key in examples:
        examples[key] = examples[key].squeeze(0)
    # if not test:
    examples["label"] = torch.FloatTensor([label])
    examples = examples.to(device)
    return examples

for split in ds:
    if split != 'test':
        ds[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines', 'authoryearofbirth', 'authoryearofdeath',
       'downloads', 'subjects', 'topic', 'Sub_A', 'Sub_B', 'Sub_C',
       'str_text_lines', 'passage_key','join_text', 'label'])
    else:
        ds[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'author_name', 'book_id', 'gutenbergbookid', 'title',
       'text', 'text_lines', 'authoryearofbirth', 'authoryearofdeath',
       'downloads', 'subjects', 'topic', 'Sub_A', 'Sub_B', 'Sub_C',
       'str_text_lines', 'passage_key','join_text', 'label'],fn_kwargs = {'test':True})

    ds[split].set_format('pt')


Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weig

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [28]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps = 1,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="f1",
    greater_is_better=True,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = 'tensorboard'
)

early_stop = EarlyStoppingCallback(1, 0.01)
tb = TensorBoardCallback()


from transformers import Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        # compute custom loss (suppose one has 3 labels with different weights)
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([1.0, 2.0, 3.0]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    

In [29]:
from sklearn.metrics import f1_score

def compute_metrics_for_classification(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape(-1, 1)
    f1 = f1_score(labels, predictions, squared=True)
    print(f"F1: {f1}")
    
    return {"F1": f1}

In [31]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["val"],
    compute_metrics=compute_metrics_for_classification,
    callbacks=[tb, early_stop]
)

You are adding a <class 'transformers.integrations.TensorBoardCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
TensorBoardCallback


In [33]:
ds['train']

Dataset({
    features: [],
    num_rows: 0
})

In [32]:
trainer.train()

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [ ]:
import gensim
embedding_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec.model')

In [ ]:
trained_weights = embedding_model.wv

In [ ]:
class DocumentAttentionClassifier(nn.Module):
    
    def __init__(self, vocab_size, embedding_size, num_heads, hidden_dim,trained_weights, n_classes):
        '''
        Creates the new classifier model. embeddings_fname is a string containing the
        filename with the saved pytorch parameters (the state dict) for the Embedding
        object that should be used to initialize this class's word Embedding parameters
        '''
        super(DocumentAttentionClassifier, self).__init__()
        
        # Save the input arguments to the state
        self.vocab_size = vocab_size
        self.embedding_size = embedding_size
        self.num_heads = num_heads
        self.embeddings_fname = vocab_size        
        
       
        # self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_dim, batch_first=True, bidirectional=True)
        self.attention = nn.Linear(hidden_dim * 2, 1)
        self.fc = nn.Linear(hidden_dim * 2, n_classes)
        # trained_weights = torch.load(embeddings_fname)['target_embeddings.weight']

        self.embeddings = nn.Embedding.from_pretrained(trained_weights, freeze = False)
        # # self.embeddings = nn.Embedding()

        # self.lstm = nn.LSTM(embedding_size, hidden_dim, bidirectional=True)
        # # self.attention = torch.rand(self.num_heads, self.embedding_size, requires_grad = True, device=device)
        # self.attention = torch.rand(self.num_heads, hidden_dim * 2, requires_grad = True, device=device)
        # self.linear = nn.Linear(num_heads * embedding_size, n_classes)
    
    def forward(self, w):
        embedded = self.embeddings(w)
        lstm_output, _ = self.lstm(w.transpose(1,2)) 
        attention_scores = self.attention(lstm_output) 
        attention_weights = F.softmax(attention_scores, dim=1)  
        context_vector = torch.sum(attention_weights * lstm_output, dim=1)  
        output = self.fc(context_vector)  # shape: (batch_size, num_classes)
        # w = w.squeeze()

        # lstm_out, _ = self.lstm(w.T)
        # # w = torch.t(self.embeddings(word_ids).squeeze()) # Embedding_Dim 
        # r = torch.matmul(self.attention, lstm_out.T)
        # a = torch.softmax(r, 1)
        # reweighted = a @ w.T
        # output = self.linear(reweighted.view(-1))

        # return torch.softmax(output, dim=0), a.T
        return output, attention_weights

In [ ]:
datasets = {}

datasets['train'] = list(zip(X_train, y_train))
datasets['val'] = list(zip(X_val, y_val))
datasets['test'] = list(zip(X_test, y_test))

train_list = datasets['train']
val_list = datasets['val']

model = DocumentAttentionClassifier(1, 50, 4, 32, trained_weights, n_classes)
model = model.to(device)

In [ ]:
def run_eval(model, eval_data, n_classes, kwargs):
    '''
    Scores the model on the evaluation data and returns the F1
    Eval Data must be in DataLoader-ready format
    '''

    eval_loader = DataLoader(eval_data, batch_size = 1, shuffle = False, collate_fn=collate_func, **kwargs)

    threshold = 0.2
    probs = np.zeros((len(eval_loader), n_classes))
    labels = []
    
    with torch.no_grad():
        for idx, x in enumerate(eval_loader):
            word_ids, label = x
            labels.append(label.cpu().numpy())
            output, weights = model(word_ids)
            probs[idx] = output.cpu().numpy()
    
    
    y_pred = np.array([np.argmax(p) for p in probs], dtype = int)
    labels = np.array(labels)

    y_true = [np.argmax(l) for l in labels]
    
    
    return labels, y_pred, f1_score(y_true, y_pred, average='micro')

In [ ]:

loss_period = 5
# model = model.to(device)
writer = SummaryWriter()
loss_function = nn.CrossEntropyLoss()

# VVV GOLD STANDARD VVV
optimizer = optim.AdamW(model.parameters(), lr = 5e-5, weight_decay = 0.1)
# ^^^ GOLD STANDARD ^^^

# optimizer = optim.AdamW(model.parameters(), lr = 5e-3, weight_decay = 0.1)

# optimizer = optim.AdamW(model.parameters())
# optimizer = optim.RMSprop(model.parameters(), 5e-3)
# optimizer = optim.SGD(model.parameters(), lr = 5e-4)

train_loader = DataLoader(train_list, batch_size=16, shuffle=True, collate_fn=collate_func, **kwargs)
n_epochs = 10
# n_epochs = 1

# + vscode={"languageId": "python"}
loss_idx = 0
loss_record = []
model.train()

# + vscode={"languageId": "python"}r
for epoch in tqdm(range(n_epochs)):

    loss_sum = 0

    for step, data in tqdm(enumerate(train_loader)):

        word_ids, labels = data
        model.train()
        model.zero_grad()
        output, weights = model(word_ids)
        loss = loss_function(output, labels.squeeze().float())
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        
        # TODO: Based on the details in the Homework PDF, periodically
        # report the running-sum of the loss to tensorboard. Be sure
        # to reset the running sum after reporting it.

        if not step % loss_period and step:
            writer.add_scalar("Loss", loss_sum, loss_idx)
            if not step % (loss_period * 10) and step:
                model.eval()
                _y, _y2, f1 = run_eval(model, val_list, n_classes, kwargs)
                writer.add_scalar("F1", f1, loss_idx)
                model.train()
            loss_record.append(loss_sum)
            loss_sum = 0
            loss_idx += 1
            

        # TODO: it can be helpful to add some early stopping here after
        # a fixed number of steps (e.g., if step > max_steps)
        

# once you finish training, it's good practice to switch to eval.
model.eval()

torch.save(optimizer.state_dict(), 'trained_opt_')
torch.save(model.state_dict(), 'trained_model_')

y_true, y_pred, f1 = run_eval(model, val_list, n_classes, kwargs)
print("F1 Score of : "+ str(f1))